In [63]:
#!pip install robobrowser tabula-py

In [113]:
import csv
import datetime
import json
import mechanize
import os
import pandas as pd
import requests
import re
import tabula
import zipfile

# https://github.com/jmcarp/robobrowser/issues/93
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property
from robobrowser import RoboBrowser

from bs4 import BeautifulSoup

## MCA Director Disqualified List

In [6]:
PARENT_SOURCE = 'http://www.mca.gov.in'
SOURCE = 'http://www.mca.gov.in/MinistryV2/disqualifieddirectorslist.html'

In [23]:
br = RoboBrowser(history=True,
                 user_agent='Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
                 parser='lxml'
                 )

In [8]:
br.open(SOURCE)

In [55]:
html = br.response.content

In [56]:
soup = BeautifulSoup(html, 'lxml')

In [57]:
links = [a['href'] for a in soup.find_all('a', class_='links_black_EES')]

In [58]:
if not os.path.exists('files'):
    os.makedirs('files')

In [59]:
file_sources = {}

In [60]:
# Download files
for link in links:
    br.open(PARENT_SOURCE + link)
    html = br.response.content
    soup = BeautifulSoup(html, 'lxml')
    file_links = [a['href'] for a in soup.find_all('a', class_='links_black_EES')]
    for file_link in file_links:
        FILE_URL = PARENT_SOURCE + file_link
        FILENAME = file_link.split('/')[-1]
        br.open(FILE_URL)
        if br.response.status_code != 200:
            break
        file_sources[FILENAME] = FILE_URL
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)

In [61]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [37]:
FILENAME = 'files/DisqualifiedDirectorsAhmedabadSearchable_02072020.pdf'

In [38]:
tables = tabula.read_pdf(FILENAME, pages='all', multiple_tables=True)

In [39]:
tables[0] # First page

,Sr No.,CIN,Company,DIN,Director,ROC Code,Status,Period\rfrom,Period Till
0,1,L11110GJ1993PLC019481,GUJARAT AQUA INDUSTRIES LTD,147642,JAYESHBHAI PATEL,RC202,Disqualified,01.11.16,31.10.21
1,2,L11110GJ1993PLC019481,GUJARAT AQUA INDUSTRIES LTD,53097,KULIN VYAS,RC202,Disqualified,01.11.16,31.10.21
2,3,L11110GJ1993PLC019481,GUJARAT AQUA INDUSTRIES LTD,6775586,ASHOKKUMAR PATEL,RC202,Disqualified,01.11.16,31.10.21
3,4,L15174GJ1990PLC014502,DARNENDRA AGRO FOODINDUSTRIES LTD,1276686,DHARMENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
4,5,L15174GJ1990PLC014502,DARNENDRA AGRO FOODINDUSTRIES LTD,1276710,NAVINCHANDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
5,6,L15174GJ1990PLC014502,DARNENDRA AGRO FOODINDUSTRIES LTD,1276729,BHUPENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
6,7,L15200GJ1986PLC009146,DHARNENDRA INDUSTRIES LIMITED,1276686,DHARMENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
7,8,L15200GJ1986PLC009146,DHARNENDRA INDUSTRIES LIMITED,1276710,NAVINCHANDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
8,9,L15200GJ1986PLC009146,DHARNENDRA INDUSTRIES LIMITED,1276729,BHUPENDRA GANDHI,RC202,Disqualified,01.11.16,31.10.21
9,10,L15400GJ1992PLC017712,JASMINA INDUSTRIES LIMITED,3298444,OBULREDDY PUPPALA,RC202,Disqualified,01.11.16,31.10.21


Fields are `Sr No.`, `CIN`, `Company`, `DIN`, `Director`, `ROC code`, `Status`, `Period from`, `Period Till`
Globally we need `PAN`, `Name`, `AddedDate` (day of blacklisting according to the source),
`Source`, `Meta` (a JSON encoded field of whatever fields each source provides).

In [1]:
tabula.convert_into(FILENAME, FILENAME.replace('pdf', 'csv'), pages='all')

In [3]:
filenames = [os.path.join('files', filename) for filename in os.listdir('files')]

In [8]:
# Parse from pdf files
for filename in filenames:
    if filename.endswith('pdf'):
        tabula.convert_into(filename, filename.replace('pdf', 'csv'), pages='all')

Got stderr: Dec 30, 2020 2:35:54 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Dec 30, 2020 2:35:55 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Dec 30, 2020 2:35:56 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>

Got stderr: Dec 30, 2020 3:18:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:18:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:18:03 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:18:03 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:18:03 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE:

Got stderr: Dec 30, 2020 3:29:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 3:29:05 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:29:06 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:29:07 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:29:07 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:29:08 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:29:08 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is 

Got stderr: Dec 30, 2020 3:30:09 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 3:30:09 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: Dec 30, 2020 3:30:20 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 3:30:21 

Got stderr: Dec 30, 2020 3:32:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 30, 2020 3:32:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Dec 30, 2020 3:32:39 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:32:39 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:32:39 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:32:39 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:32:40 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 3:32:40 PM org.apache.pdfbox.contentst

Got stderr: Dec 30, 2020 4:09:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Dec 30, 2020 4:12:19 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 4:12:19 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:19 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:19 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:20 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:20 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:20 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:20 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 2020 4:12:20 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Dec 30, 

...

Got stderr: Dec 30, 2020 5:00:36 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:00:36 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed

Got stderr: Dec 30, 2020 5:01:34 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Dec 30, 2020 5:01:35 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Dec 30, 2020 5:01:35 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:01:35 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio 

Got stderr: Dec 30, 2020 5:02:58 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:02:58 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:02:58 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:03:40 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:03:40 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:03:40 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:03:40 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: Dec 30, 2020 5:04:40 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:04:49 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: Dec 30, 2020 5:05:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Dec 30, 2020 5:05:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine 

Got stderr: Dec 30, 2020 5:06:00 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:06:00 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed

Got stderr: Dec 30, 2020 5:06:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Dec 30, 2020 5:06:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Dec 30, 2020 5:06:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:06:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio 

Got stderr: Dec 30, 2020 5:06:11 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: Dec 30, 2020 5:06:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 5:06:15 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:06:15 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed

Got stderr: Dec 30, 2020 5:15:01 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:15:01 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:15:02 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 5:15:02 PM org.apache.p

Got stderr: Dec 30, 2020 6:12:34 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Dec 30, 2020 6:12:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 30, 2020 6:13:33 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: Dec 30, 2020 6:20:09 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 6:20:10 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 6:20:10 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 6:20:10 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Dec 30, 2020 6:20:10 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbi

In [5]:
# Convert xlsx to csv
for filename in filenames:
    for ext in ['xlsx', 'xls']:
        if filename.endswith(ext):
            excel_file = pd.read_excel(filename)
            excel_file.to_csv(filename.replace(ext, 'csv'),
                              index=None,
                              header=True)

## IRDA Blacklisted

In [53]:
PARENT_SOURCE = 'https://agencyportal.irdai.gov.in'
SOURCE = 'https://agencyportal.irdai.gov.in/PublicAccess/BlackListedAgent.aspx'

In [54]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0'}

In [55]:
site_data = {'ctl00$ContentPlaceHolder1$txtStartDate': datetime.datetime.strftime(datetime.datetime.today(), '%d %b %Y'),
             'ctl00$ContentPlaceHolder1$btnExport': 'Export'}

In [56]:
with requests.Session() as session:
    response = session.get(SOURCE, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')

    site_data['__VIEWSTATE'] = soup.find('input', attrs={'id': '__VIEWSTATE'})['value']
    site_data['__VIEWSTATEGENERATOR'] = soup.find('input', attrs={'id': '__VIEWSTATEGENERATOR'})['value']
    site_data['__EVENTTARGET'] = soup.find('input', attrs={'id': '__EVENTTARGET'})['value']
    site_data['__EVENTARGUMENT'] = soup.find('input', attrs={'id': '__EVENTARGUMENT'})['value']
    #site_data['__SCROLLPOSITIONX'] = soup.find('input', attrs={'id': '__SCROLLPOSITIONX'})['value']
    #site_data['__SCROLLPOSITIONY'] = soup.find('input', attrs={'id': '__SCROLLPOSITIONY'})['value']
    site_data['__EVENTVALIDATION'] = soup.find('input', attrs={'id': '__EVENTVALIDATION'})['value']

    response = session.post(SOURCE, data=site_data, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')

In [57]:
# Parse table
table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_AgencyGrid'})
out_rows = []
out_rows.append([th.text for th in table.find_all('th')])
for row in table.find_all('tr')[1:]:
    out_row = []
    for col in row.find_all('td'):
        out_row.append(col.text)
    out_rows.append(out_row)

In [43]:
FILENAME = 'irda-blacklisted.csv'

In [44]:
# Write csv
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [45]:
file_sources = {}

In [46]:
file_sources[FILENAME] = SOURCE

In [47]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [61]:
#br = mechanize.Browser()

In [62]:
#br.addheaders = [('User-Agent', 'Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0')]

In [63]:
#br.open(SOURCE)

In [64]:
#soup = BeautifulSoup(br.response().read())

In [65]:
#br.select_form(name='aspnetForm')

In [66]:
#br.form.set_all_readonly(False)

In [266]:
#br['__VIEWSTATE'] = soup.find('input', attrs={'id': '__VIEWSTATE'})['value']

In [267]:
#br['__VIEWSTATEGENERATOR'] = soup.find('input', attrs={'id': '__VIEWSTATEGENERATOR'})['value']

In [67]:
#br['__EVENTTARGET'] = soup.find('input', attrs={'id': '__EVENTTARGET'})['value']

In [68]:
#br['__EVENTARGUMENT'] = soup.find('input', attrs={'id': '__EVENTARGUMENT'})['value']

In [69]:
#br['__EVENTVALIDATION'] = soup.find('input', attrs={'id': '__EVENTVALIDATION'})['value']

In [70]:
#br['__SCROLLPOSITIONX'] = soup.find('input', attrs={'id': '__SCROLLPOSITIONX'})['value']

In [71]:
#br['__SCROLLPOSITIONY'] = soup.find('input', attrs={'id': '__SCROLLPOSITIONY'})['value']

In [72]:
#br['ctl00$ContentPlaceHolder1$txtStartDate'] = datetime.datetime.strftime(datetime.datetime.today(), '%d+%b+%Y')

In [73]:
#br['ctl00$ContentPlaceHolder1$btnExport'] = 'Export'

In [60]:
#!curl 'https://agencyportal.irdai.gov.in/PublicAccess/BlackListedAgent.aspx' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0' -H 'Accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'-H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Content-Type:application/x-www-form-urlencoded' -H 'Origin:https://agencyportal.irdai.gov.in' -H 'Connection: keep-alive' -H 'Referer:https://agencyportal.irdai.gov.in/PublicAccess/BlackListedAgent.aspx' -H 'Cookie: ASP.NET_SessionId=eilqghogvy1umq0mg3qhslfo' -H 'Upgrade-Insecure-Requests: 1' -H 'Pragma:no-cache' -H 'Cache-Control:no-cache' --data-raw '__EVENTTARGET=&__EVENTARGUMENT=&__VIEWSTATE=%2FwEPDwUJNjA0NDA3MjkxD2QWAmYPZBYCAgcPZBYCZg9kFgICAQ9kFgICAw9kFgICAQ88KwARAgEQFgAWABYADBQrAABkGAEFJGN0bDAwJENvbnRlbnRQbGFjZUhvbGRlcjEkQWdlbmN5R3JpZA9nZPiTIyKTi5B9MsOwNXNt2EYRHwLQtFmWBZ6SZFR6iNl1&__VIEWSTATEGENERATOR=6182BDDA&__SCROLLPOSITIONX=0&__SCROLLPOSITIONY=0&__EVENTVALIDATION=%2FwEdAAVXgfRDSiPYtlyLroSkFytKVQGbFRIKjb%2FOTPkkCDIIaI5MW6XMaFjzT9nUdsIknovnmPAvtsqGge98WsH6sptEtd4irRCGYF9cWkg9N731x%2BJqr98eZ4yRCNbvVhpBNj4z7rXyU%2BYmzjQTweb8rTeD&ctl00%24ContentPlaceHolder1%24txtStartDate=13+Jan+2021&ctl00%24ContentPlaceHolder1%24btnExport=Export'

In [59]:
#br.submit()

In [58]:
#br.response().read()

## MCA Proclaimed Offenders (Ind)

In [3]:
PARENT_SOURCE = 'http://www.mca.gov.in'
SOURCE = 'http://www.mca.gov.in/MinistryV2/proclaimedoffenders.html'

In [3]:
br = RoboBrowser(history=True,
                 user_agent='Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
                 parser='lxml'
                 )

In [5]:
br.open(SOURCE)

In [6]:
html = br.response.content

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [8]:
links = [a['href'] for a in soup.find_all('a', class_='links_black_EES')]

In [9]:
if not os.path.exists('files'):
    os.makedirs('files')

In [10]:
file_sources = {}

In [12]:
# Download files
for link in links:
    br.open(PARENT_SOURCE + link)
    html = br.response.content
    soup = BeautifulSoup(html, 'lxml')
    file_links = [a['href'] for a in soup.find_all('a', class_='links_black_EES')]
    for file_link in file_links:
        FILE_URL = PARENT_SOURCE + file_link
        FILENAME = file_link.split('/')[-1]
        br.open(FILE_URL)
        if br.response.status_code != 200:
            break
        file_sources[FILENAME] = FILE_URL
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)

In [13]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [14]:
filenames = [os.path.join('files', filename) for filename in os.listdir('files')]

In [15]:
# Parse from pdf files
for filename in filenames:
    if filename.endswith('pdf'):
        tabula.convert_into(filename, filename.replace('pdf', 'csv'), pages='all')

Got stderr: Jan 07, 2021 3:17:06 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Jan 07, 2021 3:17:08 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed

Got stderr: Jan 07, 2021 3:17:20 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Jan 07, 2021 3:17:23 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Jan 07, 2021 3:17:25 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Jan 07, 2021 3:17:26 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed



In [16]:
# Convert xlsx to csv
for filename in filenames:
    for ext in ['xlsx', 'xls']:
        if filename.endswith(ext):
            excel_file = pd.read_excel(filename)
            excel_file.to_csv(filename.replace(ext, 'csv'),
                              index=None,
                              header=True)

## SFIO Convicted

In [68]:
SOURCE = 'https://sfio.nic.in/'

In [69]:
br.open(SOURCE)

In [70]:
html = br.response.content

In [71]:
soup = BeautifulSoup(html, 'lxml')

In [72]:
menu_links = {link.text : link.get('href') for link in soup.find_all('a', class_='menu__link')}

In [73]:
FILE_URL = menu_links['List of Directors Convicted']

In [74]:
FILE_URL

'https://sfio.nic.in/sites/default/files/pdf/Conviction Matters from 2003.pdf'

In [75]:
FILENAME = FILE_URL.split('/')[-1]

In [76]:
file_sources = {}

In [77]:
if not os.path.exists('files'):
    os.makedirs('files')

In [78]:
# Download file
br.open(FILE_URL)
if br.response.status_code == 200:
    file_sources[FILENAME] = FILE_URL
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)

In [79]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [82]:
# Parse from pdf
tables = tabula.read_pdf(os.path.join('files', FILENAME), pages='all', multiple_tables=True)

In [83]:
# Convert pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME.replace('pdf', 'csv')), pages='all')

## SFIO Proclaimed Offenders

In [84]:
FILE_URL = menu_links['List of Proclaimed Offenders']

In [85]:
FILE_URL

'https://sfio.nic.in/sites/default/files/pdf/Proclaimed Offenders Details DK Sabat.pdf'

In [86]:
FILENAME = FILE_URL.split('/')[-1]

In [87]:
file_sources = {}

In [88]:
if not os.path.exists('files'):
    os.makedirs('files')

In [89]:
# Download file
br.open(FILE_URL)
if br.response.status_code == 200:
    file_sources[FILENAME] = FILE_URL
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)

In [92]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [90]:
# Parse from pdf
tables = tabula.read_pdf(os.path.join('files', FILENAME), pages='all', multiple_tables=True)

In [91]:
# Convert pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME.replace('pdf', 'csv')), pages='all')

## Wildlife Crime Convicts

In [93]:
SOURCE = 'http://wccb.gov.in/Content/Convicts.aspx'

In [94]:
br.open(SOURCE)

In [95]:
html = br.response.content

In [97]:
soup = BeautifulSoup(html, 'lxml')

In [101]:
table = soup.find('table')

In [111]:
out_rows = []
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    out_row = []

    # Omit col 2 - photograph
    for col in cols:
        if cols.index(col) != 2:
            out_row.append(col.text)
    out_rows.append(out_row)

In [128]:
FILENAME = 'wildlife-crime-convicts.csv'

In [129]:
if not os.path.exists('files'):
    os.makedirs('files')

In [130]:
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [131]:
file_sources = {}

In [132]:
file_sources[FILENAME] = SOURCE 

In [133]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

## MCA Director Defaulter List

In [4]:
PARENT_SOURCE = 'http://www.mca.gov.in'
SOURCE = 'http://www.mca.gov.in/MinistryV2/defaulterdirectorslist.html'

In [5]:
br.open(SOURCE)

In [7]:
html = br.response.content

In [8]:
soup = BeautifulSoup(html, 'lxml')

In [16]:
links = [link['href'] for link in soup.find_all('a') if link.text.startswith('Link to view')]

In [18]:
if not os.path.exists('files'):
    os.makedirs('files')

In [19]:
file_sources = {}

In [20]:
# Download pdf
for link in links:
    FILENAME = link.split('/')[-1]
    FILE_URL = PARENT_SOURCE + link
    file_sources[FILENAME] = FILE_URL
    
    br.open(FILE_URL)
    if br.response.status_code == 200:
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)

In [21]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [22]:
filenames = [os.path.join('files', filename) for filename in os.listdir('files')]

In [24]:
# Parse from pdf files
for filename in filenames:
    if filename.endswith('pdf'):
        tabula.convert_into(filename, filename.replace('pdf', 'csv'), pages='all')

## MCX Secretaries Defaulter List

In [25]:
SOURCE = 'https://www.mcxindia.com/Investor-Services/defaulters/defaulters-list'

In [26]:
br.open(SOURCE)

In [27]:
html = br.response.content

In [49]:
soup = BeautifulSoup(html, 'lxml')

In [52]:
table = soup.find_all('table', id='lstDefaulter')[0]

In [111]:
out_rows = []

In [112]:
for row in table.find_all('tr'):
    cols = row.find_all('td')
    out_row = []
    for col in cols:
        out_row.append(col.text)
        if col.find('a'):
            out_row.append(col.find('a')['href'])
    if out_row[4] == '\n\n\n\n':
        out_rows.append([out_row[0], out_row[1], out_row[2], out_row[3], out_row[5]])
    else:
        out_rows.append(out_row)

In [113]:
FILENAME = 'msx-secrataries-defaulter-list.csv'

In [114]:
if not os.path.exists('files'):
    os.makedirs('files')

In [115]:
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [116]:
file_sources = {}

In [117]:
file_sources[FILENAME] = SOURCE 

In [118]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

## MCA Company Defaulters List

In [119]:
PARENT_SOURCE = 'http://www.mca.gov.in'
SOURCE = 'http://www.mca.gov.in/MinistryV2/defaultercompanieslist.html'

In [120]:
br.open(SOURCE)

In [121]:
html = br.response.content

In [122]:
soup = BeautifulSoup(html, 'lxml')

In [123]:
links = [link['href'] for link in soup.find_all('a') if link.text.startswith('Link to view')]

In [124]:
links

['/MCA21/dca/EES_Companies_List/CompanyListA_F.pdf',
 '/MCA21/dca/EES_Companies_List/CompanyListG_L.pdf',
 '/MCA21/dca/EES_Companies_List/CompanyListM_R.pdf',
 '/MCA21/dca/EES_Companies_List/CompanyListS_Z.pdf']

In [125]:
if not os.path.exists('files'):
    os.makedirs('files')

In [126]:
file_sources = {}

In [127]:
# Download pdf
for link in links:
    FILENAME = link.split('/')[-1]
    FILE_URL = PARENT_SOURCE + link
    file_sources[FILENAME] = FILE_URL
    
    br.open(FILE_URL)
    if br.response.status_code == 200:
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)

In [128]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [129]:
filenames = [os.path.join('files', filename) for filename in os.listdir('files')]

In [130]:
# Parse from pdf files
for filename in filenames:
    if filename.endswith('pdf'):
        tabula.convert_into(filename, filename.replace('pdf', 'csv'), pages='all')

Got stderr: Jan 10, 2021 2:23:04 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



## Income Tax Defaulters

In [4]:
SOURCE = 'http://office.incometaxindia.gov.in/administration/_layouts/15/inplview.aspx?List={5A26177B-D7A0-4251-843D-5E6C0B3C3DF2}&View={D8DD9754-8FD1-4D72-9908-727646E99CA0}&ViewCount=450&IsXslView=TRUE&IsCSR=TRUE&Paged=TRUE&p_ID='

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0'}

In [38]:
out_rows = [['Title', 'Approval Status', 'Address', 'Father Name', 'Date of Birth', 'PAN', 'Source of Incomeme', 'Tax Arrear', 'Year Prefix', 'CMSID', 'Category of Assessee', 'Income Tax Authority', 'Remarks', 'Sort Order', 'Active', 'Modified', 'Modified By']]

In [39]:
# Get table
for page in range(4):
    first_item_row = page * row_count + 1
    response = requests.post(SOURCE + str(first_item_row))

    data = response.json()
    
    for row in data['Row']:
        parent = BeautifulSoup(row['DITHtml'], 'lxml')
        parent = parent.find('div').text
        parent = parent.replace('<br>', '\n')
        income_tax_authority = BeautifulSoup(row['DITHtml2'], 'lxml')
        income_tax_authority = income_tax_authority.find('div').text
        income_tax_authority = income_tax_authority.replace('<br>', '\n')
        out_rows.append([row['Title'], row['_ModerationStatus'], row['Address'].replace('<br>', '\n'), parent, row['Date_x0020_of_x0020_Birth'], row['PAN'], row['Source_x0020_of_x0020_Income'], row['Tax_x0020_Arrear'], row['Year_x0020_Prefix'], row['CMSID'], row['Category_x0020_of_x0020_Assessee'], income_tax_authority, row['Remarks'], row['Sort_x0020_Order'], row['Active'], row['Modified'], row['Editor'][0]['title']])

In [46]:
FILENAME = 'income-tax-defaulters.csv'

In [47]:
FILE_URL = 'http://office.incometaxindia.gov.in/administration/Lists/Tax%20Defaulters/AllItems.aspx'

In [48]:
file_sources = {}

In [49]:
file_sources[FILENAME] = FILE_URL

In [50]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [51]:
# Write csv
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

## United Kingdom Sanction List

In [2]:
SOURCE = 'https://www.gov.uk/government/publications/the-uk-sanctions-list'

In [4]:
br = RoboBrowser(history=True,
                 user_agent='Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
                 parser='lxml'
                 )

In [5]:
br.open(SOURCE)

In [6]:
html = br.response.content

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [31]:
links = soup.find_all('a')

In [22]:
file_urls = []

In [32]:
for link in links:
    if 'ods' in link['href'] or 'odt' in link['href']:
        file_urls.append(link['href'])

In [34]:
# Remove duplicates
file_urls = list(set(file_urls))

In [35]:
file_urls

['https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/950759/UK_Sanctions_List.odt',
 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/950762/UK_Sanctions_List.ods']

In [39]:
file_sources = {}

In [40]:
if not os.path.exists('files'):
    os.makedirs('files')

In [41]:
for FILE_URL in file_urls:
    FILENAME = FILE_URL.split('/')[-1]
    br.open(FILE_URL)
    if br.response.status_code == 200:
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)
        file_sources[FILENAME] = FILE_URL

In [42]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

## BSE Defaulter and Expelled Members

In [43]:
SOURCE = 'https://www.bseindia.com/static/members/List_defaulters_Expelled_members.aspx'

In [44]:
br.open(SOURCE)

In [45]:
html = br.response.content

In [46]:
soup = BeautifulSoup(html, 'lxml')

In [47]:
table = soup.find('table')

In [61]:
out_rows = []

In [62]:
for row in table.find_all('tr'):
    out_row = []
    for col in row.find_all('td'):
        out_row.append(col.text)
    out_rows.append(out_row)

In [63]:
out_rows = out_rows[1:]

In [67]:
FILENAME = 'List_defaulters_Expelled_members.csv'

In [70]:
file_sources = {}

In [68]:
if not os.path.exists('files'):
    os.makedirs('files')

In [69]:
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [71]:
file_sources[FILENAME] = SOURCE

In [73]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

## ICEX Defaulter Members

In [4]:
SOURCE = 'https://www.icexindia.com/membership/expelled-defaulter-surrendered-members'

In [5]:
br.open(SOURCE)

In [6]:
html = br.response.content

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [13]:
soup = soup.find_all('ul', {'class': 'downloadSec'})[0]

In [17]:
links = {link.text : link.find('a').get('href') for link in soup.find_all('li')}

In [18]:
links

{'List of Surrendered Members ': 'https://www.icexindia.com/docs/default-source/list-of-surrendered-members/list-of-surrendered-membersae6b9d010302488d96f153facd6edce7.pdf?sfvrsn=57329450_2',
 'List of Defaulter Members': 'https://www.icexindia.com/docs/default-source/list-of-surrendered-members/list-of-defaulter-members.pdf?sfvrsn=1d53dcf9_12',
 'List of Expelled Members': 'https://www.icexindia.com/docs/default-source/list-of-surrendered-members/list-of-expelled-members.pdf?sfvrsn=97ffec49_10'}

In [37]:
FILE_URL = links['List of Defaulter Members']

In [38]:
FILENAME = FILE_URL.split('/')[-1]

In [39]:
FILENAME = FILENAME[:FILENAME.find('.pdf')] + '.pdf'

In [40]:
FILENAME

'list-of-defaulter-members.pdf'

In [41]:
file_sources = {}

In [42]:
file_sources[FILENAME] = FILE_URL

In [43]:
# Download pdf
br.open(FILE_URL)
if br.response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)
    with open('files/file_sources.json', 'w') as f:
        json.dump(file_sources, f)

In [45]:
# Convert pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME.replace('pdf', 'csv')), pages='all')

Got stderr: Jan 13, 2021 3:29:13 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



## ICEX Expelled Members

In [46]:
FILE_URL = links['List of Expelled Members']

In [47]:
FILENAME = FILE_URL.split('/')[-1]

In [48]:
FILENAME = FILENAME[:FILENAME.find('.pdf')] + '.pdf'

In [49]:
FILENAME

'list-of-expelled-members.pdf'

In [50]:
file_sources = {}

In [51]:
file_sources[FILENAME] = FILE_URL

In [52]:
# Download pdf
br.open(FILE_URL)
if br.response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)
    with open('files/file_sources.json', 'w') as f:
        json.dump(file_sources, f)

In [53]:
# Convert pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME.replace('pdf', 'csv')), pages='all')

Got stderr: Jan 13, 2021 3:33:28 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Jan 13, 2021 3:33:41 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



## MCX Action AP

In [34]:
SOURCE = 'https://www.mcxindia.com/membership/notice-board/notice-board-disciplinary-action'
PARENT_SOURCE = 'https://www.mcxindia.com'

In [5]:
br.open(SOURCE)

In [6]:
html = br.response.content

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [63]:
table = soup.find('table', {'class': 'table1'})
out_rows = []
for row in table.find_all('tr')[1:]:
    out_row = []
    for col in row.find_all('td'):
        if col.find('a'):
            link = PARENT_SOURCE + col.find('a')['href']
            link = link[:link.find('.pdf')+4]
            if link.endswith('pdf'):
                out_row.append(link)
        else:
            out_row.append(col.text.replace('  ', '').replace('\n', ''))
    out_rows.append(out_row)

In [58]:
links = {}

In [66]:
for out_row in out_rows:
    try:
        links[out_row[0]] = out_row[1]
    except IndexError:
        pass

In [67]:
links

{'List of Members declared Defaulters': 'https://www.mcxindia.com/docs/default-source/membership/notice-board-disciplinary-action/list-of-members-declared-defaulters72c0104757fb64e3bdfdff00007acb35.pdf',
 'List of Members Expelled': 'https://www.mcxindia.com/docs/default-source/membership/notice-board-disciplinary-action/list-of-member-expelled3f25bd4657fb64e3bdfdff00007acb35.pdf',
 'Members deactivated / suspended / debarred for serious violations': 'https://www.mcxindia.com/docs/default-source/membership/notice-board-disciplinary-action/members-deactivated-suspended-debarred-for-serious-violations.pdf',
 'Disciplinary Action taken against Authorised Persons': 'https://www.mcxindia.com/docs/default-source/membership/notice-board-disciplinary-action/list-of-authorised-persons.pdf'}

In [68]:
FILE_URL = links['Disciplinary Action taken against Authorised Persons']

In [70]:
FILENAME = FILE_URL.split('/')[-1]

In [71]:
# Download file
br.open(FILE_URL)
if br.response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)

In [72]:
file_sources = {}

In [73]:
file_sources[FILENAME] = FILE_URL

In [74]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [75]:
# Convert pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME.replace('pdf', 'csv')), pages='all')

## MCX Defaulter Members

In [76]:
FILE_URL = links['List of Members declared Defaulters']

In [77]:
FILENAME = FILE_URL.split('/')[-1]

In [78]:
# Download file
br.open(FILE_URL)
if br.response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)

In [79]:
file_sources = {}

In [80]:
file_sources[FILENAME] = FILE_URL

In [81]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [82]:
# Convert pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME.replace('pdf', 'csv')), pages='all')

Got stderr: Jan 15, 2021 5:44:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:30 PM org.apache.fontbox.ttf.GlyfCompositeDescript getPointCount
SEVERE: GlyphDescription for index 3 is null, returning 0
Jan 15, 2021 5:44:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 5:44:41 PM org.apache.fontbox.ttf.GlyfCompositeDescript getPointCount
SEVERE: GlyphDescription for index 3 is null, returning 0
Jan 15, 2021 5:44:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 15, 2021 

## MSE Arbitral Awards

In [29]:
SOURCE = 'https://www.msei.in/investors/list-of-arbitrators'

In [30]:
br.open(SOURCE)

In [31]:
html = br.response.content

In [32]:
soup = BeautifulSoup(html, 'lxml')

In [33]:
table = soup.find('table', {'class' : 'table-striped'})

In [34]:
links = []

In [35]:
for row in table.find_all('tr')[1:]:
    for col in row.find_all('td'):
        if col.find('a'):
            links.append(col.find('a')['href'])

In [36]:
links

['https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_AHMEDABAD.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_CHENNAI.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_DELHI.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_HYDERABAD.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_INDORE.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_KANPUR.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_KOLKATA.pdf',
 'https://www.msei.in/SX-Content/common/Investors/List-of-Arbitrators/2020/March/LIST_OF_ARBITRATORS_MUMBAI.pdf']

In [37]:
file_sources = {}

In [38]:
# Download pdf
for link in links:
    br.open(link)
    if br.response.status_code == 200:
        FILENAME = link.split('/')[-1]
        with open(os.path.join('files', FILENAME), 'wb') as f:
            f.write(br.response.content)
        file_sources[FILENAME] = link

In [39]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [40]:
filenames = [os.path.join('files', filename) for filename in os.listdir('files')]

In [41]:
# Parse from pdf files
for filename in filenames:
    if filename.endswith('pdf'):
        tabula.convert_into(filename, filename.replace('pdf', 'csv'), pages='all')

## NCDEX Suspended Defaulted Expelled Debarred Members

In [98]:
SOURCE = 'https://ncdex.com/suspended_member/latest_info'

In [99]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [100]:
form_data = {"draw":"2","columns[0][data]":"title","columns[0][name]":"","columns[0][searchable]":"true","columns[0][orderable]":"true","columns[0][search][value]":"","columns[0][search][regex]":"false","columns[1][data]":"file","columns[1][name]":"","columns[1][searchable]":"true","columns[1][orderable]":"false","columns[1][search][value]":"","columns[1][search][regex]":"false","order[0][column]":"0","order[0][dir]":"asc","start":"0","length":"10","search[value]":"","search[regex]":"false","_token":"nN12XA70opb1iH9HkDWdVfVfFckordXJnmJNz9Xz"}

In [101]:
cookies = {"_ga":"GA1.2.115752129.1610379486","_gat_gtag_UA_13122686_1":"1","_gid":"GA1.2.405025351.1610961674","ncdex_session":"eyJpdiI6IkxqV0N4XC8yUDZXWDlqMXZ2VUZ2ZFJBPT0iLCJ2YWx1ZSI6IjhIVFdacFpxdDZwQWswYzVWK0hVb3dcL1oxNzVrRWZGZTRkOUJEeThQU3htdTBQbWdnY003QW9sd0I0NTlVdjlhIiwibWFjIjoiY2YzYjFhMjg4ZGJhMzdmZWMwMTQ3NWViMTY5MzlhZGNhYTA0NjVjYzM0NmE2NDFlZjE1Njk3ZjhhMDcwNDQwNSJ9","XSRF-TOKEN":"eyJpdiI6IlU5MWJRbmZhemxSUk1IaUpYOUhiemc9PSIsInZhbHVlIjoiSzVWZzlzU3V0YWhHbXpNY2M5VE1DWXdvdmxkOEFXTEdhc0RUTjlMV3ZzUURiMlFwRUIrb21xalRVWVU2SDNKWSIsIm1hYyI6ImQwZDZlZjFhNGI4ZjBkZTgxMWY1Y2Q4YWNmYzkyODk2ODEwNzI5NDg5NjkwMTRkMzQ3NDMwZWRjZjViMzQ5MDEifQ=="}

In [102]:
response = requests.post(SOURCE, verify=False, headers=headers, data=form_data, cookies=cookies)

/home/navaneeth/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ncdex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [103]:
rjson = response.json()

In [104]:
rjson

{'draw': 2,
 'recordsTotal': 3,
 'recordsFiltered': 3,
 'data': [{'title': 'List of Suspended / Debarred / Expelled / Defaulter/ Ceased Members from October 2017',
   'srno': 1,
   'file': '<a id="file_alink" href="https://ncdex.com/public/uploads/membership/suspended_members/1608710089.pdf" target="_blank"><img alt="View" src="../public/images/icons/pdf.png" border="0"></a><br>(199.95 KB)',
   'id': 'XXXVIII',
   'date': '2020-12-18'},
  {'title': 'Notice issued to members (potential defaulters)',
   'srno': 2,
   'file': '<a id="file_alink" href="https://ncdex.com/downloads/Member-area/Surrender-membership/List of SCN issued to Members.pdf" target="_blank"><img alt="View" src="../public/images/icons/pdf.png" border="0"></a><br>(164.61 KB)',
   'id': 'XXIX',
   'date': '2020-10-21'},
  {'title': 'Suspended / Debarred / Expelled / Defaulter Members up to September,2017',
   'srno': 3,
   'file': '<a id="file_alink" href="https://ncdex.com/downloads/Member-area/Surrender-membership/Susp

In [105]:
a = rjson['data'][0]['file']

In [106]:
date = rjson['data'][0]['date']

In [107]:
soup = BeautifulSoup(a, 'lxml')

In [108]:
FILE_URL = soup.find('a')['href']

In [109]:
FILENAME = FILE_URL.split('/')[-1]

In [110]:
file_sources = {}

In [111]:
# Download pdf
response = requests.get(FILE_URL, verify=False, headers=headers)
if response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(response.content)
        file_sources[FILENAME] = FILE_URL

/home/navaneeth/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ncdex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [115]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [119]:
# pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME).replace('pdf', 'csv'), pages='all')

## NSE Regulatory Defaulting Clients

In [70]:
SOURCE = 'https://www.nseindia.com/regulations/exchange-defaulting-clients'

In [71]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [72]:
response = requests.get(SOURCE, headers=headers)

In [73]:
html = response.content

In [59]:
soup = BeautifulSoup(html, 'lxml')

In [60]:
FILE_URL = soup.find('a', {'class' : 'file'})['href']

In [61]:
FILENAME = FILE_URL.split('/')[-1]

In [62]:
file_sources = {}

In [63]:
# Download xlsx
br.open(FILE_URL)
if br.response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(br.response.content)
        file_sources[FILENAME] = FILE_URL

In [64]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

In [117]:
# pdf to csv
tabula.convert_into(os.path.join('files', FILENAME), os.path.join('files', FILENAME).replace('pdf', 'csv'), pages='all')

## UNSC_1267/UNSC Consolidated List

In [83]:
SOURCE = 'https://www.un.org/securitycouncil/content/un-sc-consolidated-list'

In [84]:
response = requests.get(SOURCE, headers=headers)

In [85]:
html = response.content

In [86]:
soup = BeautifulSoup(html, 'lxml')

In [104]:
FILE_URL = [link['href'] for link in soup.find_all('a', {'class' : 'documentlinks uw-link-btn'}) if 'xml' in link['href']][0]

In [105]:
FILE_URL

'https://scsanctions.un.org/resources/xml/en/consolidated.xml'

In [106]:
response = requests.get(FILE_URL, headers=headers)

In [108]:
xml = response.content

In [109]:
soup = BeautifulSoup(xml, 'lxml')

In [117]:
individuals = soup.find_all('individual')

In [158]:
out_rows = [['dataid', 'versionnum', 'first_name', 'second_name', 'third_name', 'un_list_type', 'reference_number', 'listed_on', 'comments1', 'designation', 'nationality', 'list_type', 'last_day_updated', 'individual_alias', 'individual_address', 'individual_date_of_birth', 'individual_place_of_birth', 'type_of_document', 'number', 'sort_key', 'sort_key_last_mod']]

In [130]:
individuals[0]

<individual><dataid>6908555</dataid><versionnum>1</versionnum><first_name> RI </first_name><second_name>WON HO</second_name><third_name></third_name><un_list_type>DPRK</un_list_type><reference_number>KPi.033</reference_number><listed_on>2016-11-30</listed_on><comments1>Ri Won Ho is a DPRK Ministry of State Security Official stationed in Syria supporting KOMID.
</comments1><designation><value>DPRK Ministry of State Security Official</value></designation><nationality><value>Democratic People's Republic of Korea</value></nationality><list_type><value>UN List</value></list_type><last_day_updated><value></value></last_day_updated><individual_alias><quality></quality><alias_name></alias_name></individual_alias><individual_address><country></country></individual_address><individual_date_of_birth><type_of_date>EXACT</type_of_date><date>1964-07-17</date></individual_date_of_birth><individual_place_of_birth></individual_place_of_birth><individual_document><type_of_document>Passport</type_of_docu

In [132]:
dataid = individuals[0].find('dataid').text
print(dataid)

6908555


In [159]:
for row in individuals:
    dataid = row.find('dataid').text
    versionnum = row.find('versionnum').text
    first_name = row.find('first_name').text
    try:
        second_name = row.find('second_name').text
    except:
        second_name = ''
    try:
        third_name = row.find('third_name').text
    except:
        third_name = ''
    un_list_type = row.find('un_list_type').text
    reference_number = row.find('reference_number').text
    listed_on = row.find('listed_on').text
    comments1 = row.find('comments1').text
    try:
        designation = row.find('designation').text
    except:
        designation = ''
    try:
        nationality = row.find('nationality').text
    except:
        nationality = ''
    list_type = row.find('list_type').text
    last_day_updated = row.find('last_day_updated').text
    individual_alias = row.find('individual_alias').text
    individual_address = row.find('individual_address').text
    individual_date_of_birth = row.find('individual_date_of_birth').text
    individual_place_of_birth = row.find('individual_place_of_birth').text
    try:
        type_of_document = row.find('type_of_document').text
        number = row.find('number').text
    except:
        type_of_document = ''
        number = ''
    try:
        sort_key = row.find('sort_key').text 
        sort_key_last_mod = row.find('sort_key_last_mod').text
    except AttributeError:
        sort_key = ''
        sort_key_last_mod = ''
    out_rows.append([dataid, versionnum, first_name, second_name, third_name, un_list_type, reference_number, listed_on, comments1, designation, nationality, list_type, last_day_updated, individual_alias, individual_address, individual_date_of_birth, individual_place_of_birth, type_of_document, number, sort_key, sort_key_last_mod])

In [160]:
FILENAME = 'UNSCConsolidatedList-individuals.csv'

In [162]:
file_sources = {}

In [163]:
file_sources[FILENAME] = FILE_URL

In [164]:
# save to csv
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [165]:
FILENAME = 'UNSCConsolidatedList-entities.csv'

In [169]:
out_rows = [['dataid', 'versionnum', 'first_name', 'un_list_type', 'reference_number', 'listed_on', 'comments1', 'list_type', 'last_day_updated', 'entity_alias', 'entity_address', 'sort_key', 'sort_key_last_mod']]

In [170]:
entities = soup.find_all('entity')

In [171]:
for row in entities:
    dataid = row.find('dataid').text
    versionnum = row.find('versionnum').text
    first_name = row.find('first_name').text
    un_list_type = row.find('un_list_type').text
    reference_number = row.find('reference_number').text
    listed_on = row.find('listed_on').text
    comments1 = row.find('comments1').text
    list_type = row.find('list_type').text
    last_day_updated = row.find('last_day_updated').text
    try:
        entity_alias = row.find('entity_alias').text
        entity_address = row.find('entity_address').text
    except AttributeError:
        entity_alias = ''
        entity_address = ''
    try:
        sort_key = row.find('sort_key').text 
        sort_key_last_mod = row.find('sort_key_last_mod').text
    except AttributeError:
        sort_key = ''
        sort_key_last_mod = ''
    out_rows.append([dataid, versionnum, first_name, un_list_type, reference_number, listed_on, comments1, list_type, last_day_updated, entity_alias, entity_address, sort_key, sort_key_last_mod])

In [122]:
entities[0]

<entity><dataid>6908629</dataid><versionnum>1</versionnum><first_name> PROPAGANDA AND AGITATION DEPARTMENT (PAD)</first_name><un_list_type>DPRK</un_list_type><reference_number>KPe.053 </reference_number><listed_on>2017-09-11</listed_on><comments1>The Propaganda and Agitation Department has full control over the media, which it uses as a tool to control the public on behalf of the DPRK leadership. The Propaganda and Agitation Department also engages in or is responsible for censorship by the Government of the DPRK, including newspaper and broadcast censorship.</comments1><list_type><value>UN List</value></list_type><last_day_updated><value></value></last_day_updated><entity_alias><quality></quality><alias_name></alias_name></entity_alias><entity_address><city>Pyongyang</city><country>Democratic People's Republic of Korea</country></entity_address><sort_key></sort_key><sort_key_last_mod></sort_key_last_mod></entity>

In [174]:
file_sources[FILENAME] = FILE_URL

In [175]:
# save to csv
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [176]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

## SEBI Debarred - NSE

In [36]:
SOURCE = 'https://www.nseindia.com/regulations/member-sebi-debarred-entities'

In [37]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "/*/","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [38]:
response = requests.get(SOURCE, headers=headers)

In [39]:
html = response.content

In [40]:
soup = BeautifulSoup(html, 'lxml')

In [41]:
FILE_URL = soup.find('a', {'class' : 'file--mime-application-vnd-ms-excel'})['href']

In [42]:
FILENAME = FILE_URL.split('/')[-1]

In [43]:
file_sources = {}

In [44]:
FILENAME

'prs_ra_sebi.xls'

In [46]:
# Download xls
response = requests.get(FILE_URL, headers=headers)
if response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(response.content)

    file_sources[FILENAME] = FILE_URL
    with open('files/file_sources.json', 'w') as f:
        json.dump(file_sources, f)

In [48]:
# xls to csv
excel_file = pd.read_excel(os.path.join('files', FILENAME))
excel_file.to_csv(os.path.join('files', FILENAME).replace('xls', 'csv'), index=None, header=True)

## UNSC_1988

In [21]:
SOURCE = 'https://scsanctions.un.org/taliban/'

In [22]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [23]:
response = requests.get(SOURCE, headers=headers)

In [26]:
xml = response.content

In [27]:
soup = BeautifulSoup(xml, 'lxml')

In [28]:
FILE_URL = SOURCE

In [29]:
individuals = soup.find_all('individual')

In [30]:
individuals[0]

<individual><dataid>2975591</dataid><versionnum>1</versionnum><first_name>ABDUL AZIZ</first_name><second_name>ABBASIN</second_name><un_list_type>Taliban</un_list_type><reference_number>TAi.155</reference_number><listed_on>2011-10-04</listed_on><name_original_script>عبد العزيز عباسین</name_original_script><comments1>Key commander in the Haqqani Network (TAe.012) under Sirajuddin
	Jallaloudine Haqqani (TAi.144). Taliban Shadow Governor for Orgun District, Paktika
	Province as of early 2010. Operated a training camp for non-Afghan fighters in Paktika
	Province. Has been involved in the transport of weapons to Afghanistan. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/How-we-work/Notices/View-UN-Notices-Individuals</comments1><list_type><value>UN List</value></list_type><last_day_updated><value>2013-04-22</value></last_day_updated><individual_alias><quality>Good</quality><alias_name>Abdul Aziz Mahsud</alias_name></individual_alias><individual_address><co

In [34]:
out_rows = [['dataid', 'versionnum', 'first_name', 'second_name', 'third_name', 'un_list_type', 'reference_number', 'listed_on', 'comments1', 'designation', 'nationality', 'list_type', 'last_day_updated', 'individual_alias', 'individual_address', 'individual_date_of_birth', 'individual_place_of_birth', 'type_of_document', 'number', 'sort_key', 'sort_key_last_mod']]

In [35]:
for row in individuals:
    dataid = row.find('dataid').text
    versionnum = row.find('versionnum').text
    first_name = row.find('first_name').text
    try:
        second_name = row.find('second_name').text
    except:
        second_name = ''
    try:
        third_name = row.find('third_name').text
    except:
        third_name = ''
    un_list_type = row.find('un_list_type').text
    reference_number = row.find('reference_number').text
    listed_on = row.find('listed_on').text
    comments1 = row.find('comments1').text
    try:
        designation = row.find('designation').text
    except:
        designation = ''
    try:
        nationality = row.find('nationality').text
    except:
        nationality = ''
    list_type = row.find('list_type').text
    last_day_updated = row.find('last_day_updated').text
    individual_alias = row.find('individual_alias').text
    individual_address = row.find('individual_address').text
    individual_date_of_birth = row.find('individual_date_of_birth').text
    individual_place_of_birth = row.find('individual_place_of_birth').text
    try:
        type_of_document = row.find('type_of_document').text
        number = row.find('number').text
    except:
        type_of_document = ''
        number = ''
    try:
        sort_key = row.find('sort_key').text 
        sort_key_last_mod = row.find('sort_key_last_mod').text
    except AttributeError:
        sort_key = ''
        sort_key_last_mod = ''
    out_rows.append([dataid, versionnum, first_name, second_name, third_name, un_list_type, reference_number, listed_on, comments1, designation, nationality, list_type, last_day_updated, individual_alias, individual_address, individual_date_of_birth, individual_place_of_birth, type_of_document, number, sort_key, sort_key_last_mod])

In [37]:
FILENAME = 'UNSC1988-individuals.csv'

In [38]:
file_sources = {}

In [39]:
file_sources[FILENAME] = FILE_URL

In [40]:
# save to csv
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [41]:
FILENAME = 'UNSC1988-entities.csv'

In [43]:
file_sources[FILENAME] = FILE_URL

In [44]:
entities = soup.find_all('entity')

In [45]:
entities[0]

<entity><dataid>6908451</dataid><versionnum>1</versionnum><first_name>HAJI BASIR AND ZARJMIL COMPANY HAWALA</first_name><un_list_type>Taliban</un_list_type><reference_number>TAe.014</reference_number><listed_on>2015-03-27-04:00</listed_on><name_original_script>د حاجی بصیر او ضرجمیل کمپنی حواله</name_original_script><comments1>Money service provider used by senior Taliban leaders to transfer funds
	to Taliban commanders in the region. Owned by Abdul Basir Noorzai (TAi.173). INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/How-we-work/Notices/View-UN-Notices-Entities</comments1><list_type><value>UN List</value></list_type><last_day_updated><value></value></last_day_updated><entity_alias><quality>a.k.a.</quality><alias_name>Haji Bashir and Zarjmil Hawala Company</alias_name></entity_alias><entity_alias><quality>a.k.a.</quality><alias_name>Haji Abdul Basir and Zar Jameel Hawala</alias_name></entity_alias><entity_alias><quality>a.k.a.</quality><alias_name>Ha

In [46]:
out_rows = [['dataid', 'versionnum', 'first_name', 'un_list_type', 'reference_number', 'listed_on', 'comments1', 'list_type', 'last_day_updated', 'entity_alias', 'entity_address', 'sort_key', 'sort_key_last_mod']]

In [47]:
for row in entities:
    dataid = row.find('dataid').text
    versionnum = row.find('versionnum').text
    first_name = row.find('first_name').text
    un_list_type = row.find('un_list_type').text
    reference_number = row.find('reference_number').text
    listed_on = row.find('listed_on').text
    comments1 = row.find('comments1').text
    list_type = row.find('list_type').text
    last_day_updated = row.find('last_day_updated').text
    try:
        entity_alias = row.find('entity_alias').text
        entity_address = row.find('entity_address').text
    except AttributeError:
        entity_alias = ''
        entity_address = ''
    try:
        sort_key = row.find('sort_key').text 
        sort_key_last_mod = row.find('sort_key_last_mod').text
    except AttributeError:
        sort_key = ''
        sort_key_last_mod = ''
    out_rows.append([dataid, versionnum, first_name, un_list_type, reference_number, listed_on, comments1, list_type, last_day_updated, entity_alias, entity_address, sort_key, sort_key_last_mod])

In [48]:
# save to csv
with open(os.path.join('files', FILENAME), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(out_rows)

In [49]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)

## SEBI Debarred - BSE

In [3]:
SOURCE = 'https://www.bseindia.com/investors/debent.aspx'
PARENT_SOURCE = 'https://www.bseindia.com/investors/'

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [5]:
response = requests.get(SOURCE, headers=headers)

In [7]:
html = response.content

In [8]:
soup = BeautifulSoup(html, 'lxml')

In [19]:
link = soup.find('div', {'id': 'divid'}).find('div')['ng-include'].replace("'", "")

In [21]:
response = requests.get(PARENT_SOURCE + link, headers=headers)

In [22]:
html = response.content

In [23]:
soup = BeautifulSoup(html, 'lxml')

In [31]:
FILE_URL = soup.find_all('a')[0]['href'][2:]

In [33]:
FILE_URL = 'https://www.bseindia.com' + FILE_URL

In [45]:
file_sources = {}

In [35]:
FILENAME = FILE_URL.split('/')[-1]

In [46]:
# Download zip
response = requests.get(FILE_URL, headers=headers)
if response.status_code == 200:
    with open(os.path.join('files', FILENAME), 'wb') as f:
        f.write(response.content)
    file_sources[FILENAME] = FILE_URL

In [40]:
# Extract zip
with zipfile.ZipFile(os.path.join('files', FILENAME), 'r') as f:
    f.extractall('files')
    FILENAME = f.namelist()[0]

In [44]:
# xlsx to csv
excel_file = pd.read_excel(os.path.join('files', FILENAME))
excel_file.to_csv(os.path.join('files', FILENAME).replace('xlsx', 'csv'), index=None, header=True)

In [48]:
with open('files/file_sources.json', 'w') as f:
    json.dump(file_sources, f)